In [21]:
import pandas as pd
import ast  

In [18]:
# 读取两个文件
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

# 合并文件
combined_df = pd.concat([df1, df2])

# 保存合并后的文件
combined_df.to_csv('porto.csv', index=False)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df2.iloc[0]

TRIP_ID                                                        T1
CALL_TYPE                                                       B
ORIGIN_CALL                                                   NaN
ORIGIN_STAND                                                 15.0
TAXI_ID                                                  20000542
TIMESTAMP                                              1408039037
DAY_TYPE                                                        A
MISSING_DATA                                                False
POLYLINE        [[-8.585676,41.148522],[-8.585712,41.148639],[...
Name: 0, dtype: object

In [ ]:
df1.iloc[0]

TRIP_ID                                       1372636858620000589
CALL_TYPE                                                       C
ORIGIN_CALL                                                   NaN
ORIGIN_STAND                                                  NaN
TAXI_ID                                                  20000589
TIMESTAMP                                              1372636858
DAY_TYPE                                                        A
MISSING_DATA                                                False
POLYLINE        [[-8.618643,41.141412],[-8.618499,41.141376],[...
Name: 0, dtype: object

In [ ]:
df=pd.read_csv('porto.csv')

/tmp/ipykernel_1074292/515716834.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('porto.csv')


In [ ]:
len(df)

1710990

In [ ]:
taxi_id_counts = df['TAXI_ID'].value_counts()
taxi_id_counts

TAXI_ID
20000080    10746
20000403     9239
20000066     8449
20000364     7821
20000483     7730
            ...  
20000264        2
20000911        2
20000940        1
20000970        1
20000931        1
Name: count, Length: 448, dtype: int64

In [ ]:
df.iloc[0]

TRIP_ID                                       1372636858620000589
CALL_TYPE                                                       C
ORIGIN_CALL                                                   NaN
ORIGIN_STAND                                                  NaN
TAXI_ID                                                  20000589
TIMESTAMP                                              1372636858
DAY_TYPE                                                        A
MISSING_DATA                                                False
POLYLINE        [[-8.618643,41.141412],[-8.618499,41.141376],[...
Name: 0, dtype: object

In [19]:
# 读取CSV文件，仅保留需要的列
df = pd.read_csv('porto.csv', usecols=['TAXI_ID', 'TIMESTAMP', 'POLYLINE'])

In [22]:
df['POLYLINE'] = df['POLYLINE'].apply(ast.literal_eval)
# 定义函数提取起点和终点，包括计算终点的时间戳
def extract_od(polyline, start_time):
    if polyline and len(polyline) >= 2:
        duration = (len(polyline) - 1) * 15  # 由于每个点代表15秒
        end_time = start_time + duration
        return [(polyline[0][0], polyline[0][1], start_time), 
                (polyline[-1][0], polyline[-1][1], end_time)]
    return None

# 应用函数并创建新列
df['OD'] = df.apply(lambda row: extract_od(row['POLYLINE'], row['TIMESTAMP']), axis=1)

# 剔除OD为空的行
df = df.dropna(subset=['OD'])

# 按TAXI_ID分组，并将所有OD串联起来
grouped = df.groupby('TAXI_ID')['OD'].apply(lambda x: [item for sublist in x for item in sublist]).reset_index()

In [24]:
len(grouped.iloc[0]['OD'])

5472

In [30]:
grouped.iloc[0]

TAXI_ID                                             20000001
OD         [(-8.584353, 41.163174, 1372662403, 1830446), ...
Name: 0, dtype: object

In [26]:
len(grouped)

442

In [27]:
unique_coords = set()
for od_list in grouped['OD']:
    for od in od_list:
        unique_coords.add((od[0], od[1]))  # 只提取经纬度

# 为每个独特的经纬度对分配一个唯一编号
coord_to_pid = {coord: pid for pid, coord in enumerate(unique_coords)}



In [31]:
coord_to_pid

{(-8.60976, 41.147748): 0,
 (-8.660799, 41.168259): 1,
 (-8.626779, 41.177106): 2,
 (-8.611065, 41.163165): 3,
 (-8.612037, 41.150583): 4,
 (-8.59968, 41.153715): 5,
 (-8.586828, 41.170041): 6,
 (-8.675163, 41.155038): 7,
 (-8.610894, 41.149431): 8,
 (-8.555508, 41.161482): 9,
 (-8.653653, 41.185611): 10,
 (-8.610849, 41.145165): 11,
 (-8.625672, 41.149908): 12,
 (-8.607258, 41.153373): 13,
 (-8.58294, 41.182029): 14,
 (-8.668827, 41.176386): 15,
 (-8.612055, 41.166504): 16,
 (-8.631855, 41.184972): 17,
 (-8.582256, 41.160519): 18,
 (-8.667891, 41.162922): 19,
 (-8.631261, 41.16555): 20,
 (-8.654616, 41.17356): 21,
 (-8.609616, 41.16141): 22,
 (-8.578719, 41.158539): 23,
 (-8.612307, 41.145849): 24,
 (-8.552394, 41.218164): 25,
 (-8.611911, 41.155551): 26,
 (-8.639595, 41.168052): 27,
 (-8.604963, 41.146686): 28,
 (-8.669799, 41.237595): 29,
 (-8.604711, 41.154732): 30,
 (-8.648451, 41.17032): 31,
 (-8.6634, 41.184531): 32,
 (-8.583039, 41.163561): 33,
 (-8.628885, 41.180643): 34,
 (-8

In [29]:
# 更新OD列，为每个经纬度对添加PID
def add_pid_to_od(od_list):
    return [(lat, lon, timestamp, coord_to_pid[(lat, lon)]) for lat, lon, timestamp in od_list]

grouped['OD'] = grouped['OD'].apply(add_pid_to_od)

In [32]:
# 创建一个新的DataFrame来存储签到数据
sign_in_data = []

# 遍历原始DataFrame
for index, row in grouped.iterrows():
    taxi_id = row['TAXI_ID']
    for od in row['OD']:
        lat, lon, timestamp, pid = od
        sign_in_data.append([taxi_id, lat, lon, timestamp, pid])

# 转换为新的DataFrame
sign_in_df = pd.DataFrame(sign_in_data, columns=['TAXI_ID', 'Latitude', 'Longitude', 'Timestamp', 'PID'])

In [33]:
# 保存到CSV文件，不包含表头和索引
sign_in_df.to_csv('porto_checkin.csv', index=False, header=False)

In [34]:
# 保存到TXT文件，不包含表头和索引，使用逗号分隔
sign_in_df.to_csv('porto_checkin.txt', index=False, header=False, sep='\t')